<a href="https://colab.research.google.com/github/GUNAPILLCO/CEIA-GdP/blob/main/3_Desarrollo/2_obtencion_preparacion_exploracion_datos/2_4_alpha_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.4. Generación de alpha factors y selección de variables

## 1. Importación de Librerías

In [49]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
import requests
warnings.filterwarnings('ignore')

## 2. Carga del dataset mnq_intraday_data

In [ ]:
def cargar_df():
  # URL del archivo raw en GitHub
  url = 'https://raw.githubusercontent.com/GUNAPILLCO/CEIA-GdP/main/3_Desarrollo/2_obtencion_preparacion_exploracion_datos/mnq_intraday_data.parquet'

  # Descargar el archivo
  response = requests.get(url)

  # Guardar el archivo en el entorno de Colab
  file_path = '/content/mnq_intraday_data.parquet'
  with open(file_path, 'wb') as f:
      f.write(response.content)

  # Leer el archivo Parquet con pandas
  df = pd.read_parquet(file_path)

  return df

In [ ]:
mnq_intraday = cargar_df()
# Asegurar que el índice esté en formato datetime (por si acaso)
mnq_intraday.index = pd.to_datetime(mnq_intraday.index)
# Crear una columna 'date' a partir del índice
mnq_intraday['date'] = mnq_intraday.index.date

#Imprimo el dataset
mnq_intraday

,open,high,low,close,volume,date
datetime,,,,,,
2019-12-23 08:30:00-05:00,8736.25,8737.50,8736.25,8737.00,30,2019-12-23
2019-12-23 08:31:00-05:00,8737.00,8737.25,8736.00,8736.50,45,2019-12-23
2019-12-23 08:32:00-05:00,8736.50,8736.75,8735.50,8736.00,73,2019-12-23
2019-12-23 08:33:00-05:00,8736.50,8736.75,8736.00,8736.75,51,2019-12-23
2019-12-23 08:34:00-05:00,8736.75,8739.00,8736.75,8737.75,91,2019-12-23
...,...,...,...,...,...,...
2024-12-27 15:56:00-05:00,21677.25,21681.75,21663.25,21678.75,5045,2024-12-27
2024-12-27 15:57:00-05:00,21679.50,21684.25,21669.00,21682.25,2826,2024-12-27
2024-12-27 15:58:00-05:00,21682.25,21694.00,21679.25,21691.25,2770,2024-12-27


## 3. Agregar Alpha Factors

### 3.1. De hipótesis 1: Momentum factor

In [ ]:
def add_momentum_factor(df: pd.DataFrame, window: int = 30, price_col: str = 'close') -> pd.DataFrame:

    ## De la hipotesis 1  momentum_z_30

    """
    Agrega un alpha factor de momentum normalizado (z-score), basado en log-retornos pasados
    con una ventana deslizante por jornada. La columna resultante se llama 'momentum_factor'.

    Este factor representa la hipótesis de continuación del momentum intradía.

    Parámetros:
    - df: DataFrame con columnas 'date' y la columna de precios.
    - window: Minutos hacia atrás para calcular el log-retorno (default = 30).
    - price_col: Columna del precio (default = 'close').

    Retorna:
    - DataFrame con una nueva columna: 'momentum_factor'.
    """

    df = df.copy()

    df['momentum'] = df.groupby('date')[price_col].transform(lambda x: np.log(x) - np.log(x.shift(window)))

    df['momentum_factor'] = df.groupby('date')['momentum'].transform(lambda x: (x - x.mean()) / x.std())

    df = df.drop(columns=['momentum'])

    return df

In [ ]:
mnq_intraday = add_momentum_factor(mnq_intraday, window=30)

### 3.2. De hipótesis 2: Mean Reversion Factor

In [ ]:
def add_mean_reversion_factor(df: pd.DataFrame, window: int = 5, price_col: str = 'close') -> pd.DataFrame:
    ## De la hipotesis 2  zcore_5

    """
    Agrega al DataFrame una única columna con el alpha factor de reversión a la media
    basado en z-score, calculado sobre una media móvil simple por jornada.

    La columna resultante se llama: 'mean_reversion_factor'.

    Parámetros:
    - df: DataFrame con columnas 'date' y 'close'.
    - window: Ventana de minutos para la media móvil (default=5).
    - price_col: Nombre de la columna de precios (default='close').

    Retorna:
    - DataFrame con una nueva columna: 'mean_reversion_factor'.
    """

    df = df.copy()

    # Función auxiliar para calcular el z-score rolling por día
    def zscore_rolling(x):
        mean = x.rolling(window=window).mean()
        std = x.rolling(window=window).std()
        return (x - mean) / std

    # Aplicar por jornada
    df['mean_reversion_factor'] = df.groupby('date')[price_col].transform(zscore_rolling)

    return df


In [ ]:

mnq_intraday = add_mean_reversion_factor(mnq_intraday, window=5)


### 3.3. De hipótesis 3: Momentum volumen factor


In [ ]:
def add_mom_vol_factor(df: pd.DataFrame, window: int = 30,
                       price_col: str = 'close', volume_col: str = 'volume',
                       date_col: str = 'date') -> pd.DataFrame:
    """
    Agrega un alpha factor basado en momentum x volumen relativo, normalizado por jornada.
    Diseñado para datasets intradía con estructura diaria independiente.

    Parámetros:
    - df: DataFrame original con columnas de precios, volumen y fecha.
    - window: Ventana (en minutos) para cálculo del momentum y volumen promedio.
    - price_col: Columna del precio (default = 'close').
    - volume_col: Columna del volumen (default = 'volume').
    - date_col: Columna que indica la jornada (default = 'date').

    Retorna:
    - DataFrame con columna adicional: 'mom_vol_z_{window}'
    """
    df = df.copy()

    momentum_col = f'momentum_{window}'
    vol_avg_col = f'vol_avg_{window}'
    vol_ratio_col = f'vol_ratio_{window}'
    factor_col = f'mom_vol_{window}'
    z_col = f'mom_vol_z_{window}'

    # Calcular momentum por día: log-retorno contra valor N minutos atrás
    df[momentum_col] = df.groupby(date_col)[price_col].transform(
        lambda x: np.log(x) - np.log(x.shift(window))
    )

    # Volumen promedio (rolling por día)
    df[vol_avg_col] = df.groupby(date_col)[volume_col].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

    # Ratio volumen actual vs promedio
    df[vol_ratio_col] = df[volume_col] / df[vol_avg_col]

    # Alpha factor (con reversión)
    df[factor_col] = -1 * df[momentum_col] * df[vol_ratio_col]

    # Normalización por día (z-score)
    df[z_col] = df.groupby(date_col)[factor_col].transform(
        lambda x: (x - x.mean()) / x.std()
    )

    return df

In [ ]:
mnq_intraday = add_mom_vol_factor(mnq_intraday, window=30)
